In [1]:
import pandas as pd
import random
from faker import Faker

# CONSTANTS SECTION
# M: Male, F: Female, B: Bisexual, N : Non Binary, A: All
GENDER_MALE, GENDER_FEMALE, GENDER_NON_BINARY, = "M", "F", "N"
SEARCHING_GENDER = ["M", "F", "B", "N", "A"]
MIN_AGE, MAX_AGE = 18, 40
INTEREST_MIN_SIZE, INTEREST_MAX_SIZE = 3, 6
DS_SIZE = 5_000

faker = Faker()

# load datasets 
# taken from https://simplemaps.com/data/us-cities
cities_df = pd.read_csv("./us_cities.csv")
interests = pd.read_csv("./interests.csv")['interest'].tolist()

ds_list = []

for i in range(DS_SIZE):
    row = {}


    choice = random.randrange(1, 4)

    if choice == 1:
        row["name"] = faker.first_name_male()
        row["surname"] = faker.last_name_male()
        row["gender"] = GENDER_MALE
    elif choice == 2:
        row["name"] = faker.first_name_female()
        row["surname"] = faker.last_name_female()

        row["gender"] = GENDER_FEMALE
    else:
        row["name"] = faker.first_name_nonbinary()
        row["surname"] = faker.last_name_nonbinary()
        row["gender"] = GENDER_NON_BINARY

    row["searching_gender"] = random.choice(SEARCHING_GENDER)
    row["age"] = random.randrange(MIN_AGE, MAX_AGE)
    row_interests_size = random.randrange(INTEREST_MIN_SIZE, INTEREST_MAX_SIZE)

    row['interests'] = ', '.join(random.sample(interests, row_interests_size))
    cities_random_idx = random.randrange(cities_df.index.start, cities_df.index.stop)
    row['latitude'] = cities_df['lat'][cities_random_idx]
    row['longitude'] = cities_df['lng'][cities_random_idx]

    ds_list.append(row)

random.shuffle(ds_list)
df = pd.DataFrame(ds_list)

print(f"Gender distribution: \n{df['gender'].value_counts(normalize=True) * 100}")
print(f"Searching Gender distribution: \n{df['searching_gender'].value_counts(normalize=True) * 100}")

df.to_csv('dataset.csv', index=False, header=True)

Gender distribution: 
N    34.12
M    33.20
F    32.68
Name: gender, dtype: float64
Searching Gender distribution: 
M    20.68
F    20.34
N    19.98
B    19.82
A    19.18
Name: searching_gender, dtype: float64
